In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
from multiversx_sdk import Address, TokenPayment, ProxyNetworkProvider
from multiversx_sdk.core.transaction_builders import ContractCallBuilder
from multiversx_sdk.core.transaction_builders import DefaultTransactionBuildersConfiguration
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(chain_id=env.CHAIN_ID)
proxy = ProxyNetworkProvider(env.PROXY_URL)

In [ ]:
contract_address = Address.new_from_bech32("erd1qqqqqqqqqqqqqpgq38cmvwwkujae3c0xmc9p9554jjctkv4w2jps83r492")

In [ ]:
from multiversx_sdk import AccountNonceHolder, UserSigner
from pathlib import Path

user = env.USER1
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))

In [ ]:
from multiversx_sdk import TransactionsConverter

transfers = [
    TokenPayment.meta_esdt_from_amount("ITHEGLDD-9089f0", 1, 5000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="redeem",
    caller=user,
    call_arguments=[],
    gas_limit=10000000,
    esdt_transfers=transfers
)

tx = builder.build()

converter = TransactionsConverter()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
from multiversx_sdk import TransactionComputer

tx.nonce = user_nonce_holder.get_nonce_then_increment()

tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)